In [1]:
push!(LOAD_PATH, "../../julia");
Base.load_path();

In [2]:
using ExtXYZ
using DelimitedFiles
using Polyhedra
using LinearAlgebra
using QHull

┌ Info: Installing scipy.spatial via the Conda scipy package...
└ @ PyCall /home/aarifi/.julia/packages/PyCall/ygXW2/src/PyCall.jl:719
┌ Info: Running `conda install -y scipy` in root environment
└ @ Conda /home/aarifi/.julia/packages/Conda/x2UxR/src/Conda.jl:127


done
Solving environment: ...working... 

done



## Package Plan ##

  environment location: /home/aarifi/.julia/conda/3

  added / updated specs:
    - scipy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scipy-1.9.3                |  py310hdfbd76f_2        26.2 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        26.2 MB

The following NEW packages will be INSTALLED:

  scipy              conda-forge/linux-64::scipy-1.9.3-py310hdfbd76f_2 



scipy-1.9.3          | 26.2 MB   |            |   0% 

scipy-1.9.3          | 26.2 MB   |            |   0% 

...

scipy-1.9.3          | 26.2 MB   | #########6 |  96% 

scipy-1.9.3          | 26.2 MB   | #########8 |  99% 


Preparing transaction: ...working... done
Verifying transaction: ...working... 

done
Executing transaction: ...working... 

done


Download Files

In [ ]:
# https://people.sc.fsu.edu/~jburkardt/datasets/sphere_lebedev_rule/sphere_lebedev_rule.html
for i = 1:140
    try
        url = "https://people.sc.fsu.edu/~jburkardt/datasets/sphere_lebedev_rule/lebedev_"*lpad(string(i), 3, "0")*".txt"
        download(url, "./Lebedev_xyz/"*string(i)*".xyz")
    catch
        continue
    end
end

Read and Store Data into Array-Format

In [ ]:
xyzDir = readdir("./Lebedev_xyz")

# structure of xyz
# theta phi w

function arrIt(arr)
    stringed = String[]
    for (i, nestedArr) in enumerate(arr)
        tempString = ""
        push!(stringed, mapreduce(x->(string(x)*", "), *,nestedArr))
    end
    return stringed
end

function writeidp(name::String, arg::Vector{String}, trafo::Function)
    numArg = size(arg, 1)
    outputStream = open("./lebedev_" * name * ".idp", "w+")
    for xyzStr in xyzDir
        numStr = SubString(xyzStr, 1:length(xyzStr)-4)
        if parse(Int, numStr) == 47
            arrString = String[]
            matrix = readdlm("./Lebedev_xyz/"*xyzStr)
            I, J = size(matrix);
            
            for (indArg, elem) in enumerate(arg)
                arrString = push!(arrString, "real[int] lebedev" * elem * numStr * " = [")
            end
            cnt = 1
            for i = 1:I
                toWrite = arrIt(trafo(matrix[i, :]...))
                for (indArg, elem) in enumerate(toWrite)
                    arrString[indArg] = arrString[indArg] * reduce(*, elem)
                end
            end
            for indArg = 1:numArg
                arrString[indArg] = arrString[indArg][1:end-2] * "];"
                write(outputStream, arrString[indArg] * "\n")
            end
        end
    end
    close(outputStream)
end



In [ ]:
function trafo1(theta, phi, w)
    return [[theta], [phi], [w]];
end

function trafo2(theta, phi, w)
    return [[theta, phi], [w]];
end

function trafo3(theta, phi, w)
    theta = theta * 2*pi/360;
    phi = phi * 2*pi/360;
    x = cos(theta) * sin(phi)
    y = sin(theta) * sin(phi)
    z = cos(phi)
    return [[x], [y], [z], [w]];
end

writeidp("x_y_z_w", ["x","y","z","w"], trafo3)

Read and Store Data into Matrix-Format (delim = "\s")

In [ ]:
# in rad!!!
xyzDir = readdir("./Lebedev_xyz")

function trafoSphere(theta, phi, w)
    theta = deg2rad(theta);
    phi = deg2rad(phi);
    x = cos(theta) * sin(phi)
    y = sin(theta) * sin(phi)
    z = cos(phi)
    return [x, y, z, w]
end

function writeMatrix(numArg, trafo, matrix)
    matrixOut = Array{Float64}(undef, 0, numArg)
    for row in eachrow(matrix)
        matrixOut = [matrixOut; trafoSphere(row...)']
    end
    return matrixOut
end


for xyzStr in xyzDir
    numStr = SubString(xyzStr, 1:length(xyzStr)-4)
    if parse(Int, numStr) > -1
        matrix = readdlm("./Lebedev_xyz/"*xyzStr)
        outputStream = open("./Lebedev_xyzw/" * string(size(matrix, 1)) * ".xyzw", "w+")
        matrixOut = writeMatrix(4, trafoSphere, matrix)

        write(outputStream, string(size(matrixOut, 1)*size(matrixOut, 2))*"\n")
        for col in eachcol(matrixOut)
            strCol = map(x->"\t"*string(x), col)
            write(outputStream, reduce(*, strCol))
            write(outputStream, "\n")
        end
        close(outputStream)
    end
end

In [3]:
xyzDir = sort(readdir("./Lebedev_xyz"));

In [12]:
# in rad!!!
xyzDir = readdir("./Lebedev_xyz"); 
xyzDirReal = map(x->parse(Float64, x[1:end-4]), xyzDir);
perm = sortperm(xyzDirReal)
xyzDir = xyzDir[perm]

cat!(a, b) = reshape(append!(vec(a), vec(b)), size(a)[1:end-1]..., :)

function trafoSphere(theta, phi, w)
    theta = deg2rad(theta);
    phi = deg2rad(phi);
    x = cos(theta) * sin(phi)
    y = sin(theta) * sin(phi)
    z = cos(phi)
    return [x, y, z]
end

function trafoSphereW(theta, phi, w)
    theta = deg2rad(theta);
    phi = deg2rad(phi);
    x = cos(theta) * sin(phi)
    y = sin(theta) * sin(phi)
    z = cos(phi)
    return [x, y, z, w]
end

function writeMatrix(numArg, trafo, matrix)
    matrixOut = Array{Float64}(undef, 0, numArg)
    for row in eachrow(matrix)
        matrixOut = [matrixOut; trafo(row...)']
    end
    return matrixOut
end

function writeRows(outputStream, matrix, delim, int)
    for row in eachrow(matrix)
        if int != -1
            strRow = map(x->string(x)*delim, row)
            strRow = push!(strRow, string(int))
        else
            strRow = map(x->string(x)*delim, row)
            strRow = strRow[1:end-1];
            push!(strRow, string(Int(row[end]))*delim)
        end
        write(outputStream, reduce(*, strRow))
        write(outputStream, "\n")
    end
end

#https://iq.opengenus.org/orientation-of-three-ordered-points/#:~:text=Orientation%20of%20three%20ordered%20points%20refers%20to%20the%20different%20ways,i.e.%20clockwise%20and%20anti%2Dclockwise.
function orientation(p1, p2, p3)
    return -sign(×(p1 - p3, p2 - p3) ⋅ p3)
end

preInt = 10000;
for xyzStr in xyzDir
    numStr = SubString(xyzStr, 1:length(xyzStr)-4)
    if parse(Int, numStr) > -1
        
        matrix = readdlm("./Lebedev_xyz/"*xyzStr)

        newInt =size(matrix, 1)

        outputStream = open("./Lebedev_meshS/" *  string(newInt) * ".mesh", "w+")

        arrNum = (preInt:1:(newInt+preInt-1))

        matrix = matrix[:, 1:2]

        matrix = cat!(matrix, arrNum');

        preInt += newInt

        matrixVertices = writeMatrix(3, trafoSphere, matrix)

        matrixVerticesW = writeMatrix(4, trafoSphereW, matrix)
        
        write(outputStream,
        """
        MeshVersionFormatted 2
        
        Dimension 3

        Vertices
        """
        )
        write(outputStream, string(size(matrixVertices, 1))*"\n")

        writeRows(outputStream, matrixVerticesW, " ", -1)

        write(outputStream, "\nTriangles\n")

        ch = chull(matrixVertices)

        matrixSimplices = Array{Int64}(undef, 0, 3)
        for (idx, elem) in enumerate(ch.simplices)
            p1 = matrixVertices[elem[1], :]
            p2 = matrixVertices[elem[2], :]
            p3 = matrixVertices[elem[3], :]

            if orientation(p1, p2, p3) > 0
                matrixSimplices = [matrixSimplices; [elem[1], elem[2], elem[3]]']
            else
                matrixSimplices = [matrixSimplices; [elem[2], elem[1], elem[3]]']
            end
        end

        write(outputStream, string(size(matrixSimplices,1))*"\n")

        writeRows(outputStream, matrixSimplices, " ", 0)

        write(outputStream, "\nEdges\n")

        matrixEdges= Array{Int64}(undef, 0, 2)
        for elem in ch.simplices
            for i = 1:3
                matrixEdges = [matrixEdges; [elem[i], elem[i%3+1]]']
            end
        end
        
        unique(matrixEdges, dims = 1)

        write(outputStream, string(size(matrixEdges,1))*"\n")

        writeRows(outputStream, matrixEdges, " ", 0)

        write(outputStream, "\nEnd\n")

        close(outputStream)
    end
end